In [1]:
import requests 
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 

import os

In [2]:
df_cc = pd.read_csv("covid_19_data.csv")
df_cc

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
10041,10042,03/29/2020,Wyoming,US,2020-03-29 23:14:06,86.0,0.0,0.0
10042,10043,03/29/2020,Xinjiang,Mainland China,2020-03-29 23:14:06,76.0,3.0,73.0
10043,10044,03/29/2020,Yukon,Canada,2020-03-29 23:14:06,4.0,0.0,0.0
10044,10045,03/29/2020,Yunnan,Mainland China,2020-03-29 23:14:06,180.0,2.0,172.0


In [3]:
df_cc = df_cc.rename(columns={'Country/Region':'Country'})
df_cc = df_cc.rename(columns={'ObservationDate':'Date'})
df_cc.head()



,SNo,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [4]:
df_countries = df_cc.groupby(['Country', 'Date']).max().reset_index().sort_values('Date', ascending=False)
df_countries = df_countries.drop_duplicates(subset = ['Country'])
df_countries = df_countries[df_countries['Confirmed']>0]

df_countries



,Country,Date,SNo,Confirmed,Deaths,Recovered
1149,Djibouti,03/29/2020,9778,18.0,0.0,0.0
173,Argentina,03/29/2020,9742,745.0,19.0,72.0
4939,Ukraine,03/29/2020,9900,475.0,10.0,6.0
3594,Peru,03/29/2020,9861,852.0,18.0,16.0
1100,Czech Republic,03/29/2020,9775,2817.0,16.0,11.0
...,...,...,...,...,...,...
4388,St. Martin,03/09/2020,4412,2.0,0.0,0.0
5035,Vatican City,03/09/2020,4507,1.0,0.0,0.0
3746,Republic of Ireland,03/08/2020,4067,21.0,0.0,0.0
0,Azerbaijan,02/28/2020,2664,1.0,0.0,0.0


In [5]:
fig = px.pie(df_countries, values = 'Confirmed',names='Country', height=600)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))

fig.show()

In [12]:
fig_2 = go.Figure(data=go.Choropleth(
    locations = df_countries['Country'],
    locationmode = 'country names',
    z = df_countries['Confirmed'],
    colorscale = 'Reds',
    marker_line_color = 'black',
    marker_line_width = 0.5,
))

fig_2.update_layout(
    title_text = 'Confirmed Cases as of March 29, 2020',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)




In [15]:
df_countrydate = df_countries[df_countries['Confirmed']>0]
df_countrydate = df_countrydate.groupby(['Date','Country']).sum().reset_index()
df_countrydate

,Date,Country,SNo,Confirmed,Deaths,Recovered
0,02/28/2020,Azerbaijan,2664,1.0,0.0,0.0
1,02/28/2020,North Ireland,2685,1.0,0.0,0.0
2,03/08/2020,Republic of Ireland,4067,21.0,0.0,0.0
3,03/09/2020,Palestine,4322,22.0,0.0,0.0
4,03/09/2020,St. Martin,4412,2.0,0.0,0.0
...,...,...,...,...,...,...
198,03/29/2020,Venezuela,9904,119.0,2.0,39.0
199,03/29/2020,Vietnam,9905,188.0,0.0,25.0
200,03/29/2020,West Bank and Gaza,9906,109.0,1.0,18.0
201,03/29/2020,Zambia,9907,29.0,0.0,0.0


In [13]:
fig3 = px.choropleth(df_countrydate, 
                    locations="Country", 
                    locationmode = "country names",
                    color="Confirmed", 
                    hover_name="Country", 
                    animation_frame="Date"
                   )

fig3.update_layout(
    title_text = 'Spread of Coronavirus',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig3.show()